# Welcome to our script collection!!

This notebook will walk you through the steps of writing as many scripts as you want and eventually download your work for submission. You can run each cell by moving your mouse to the top left of the cell and clickin on the 'play' icon that appears. Enjoy scripting and feel free to ask us any questions!

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import shutil
import json
import numpy as np
import sys
import random
from copy import deepcopy
import multiprocessing
from ProgramExecutor import read_program
from GraphReader import GraphReader, init_graph_file, scene_num
from ProgramExecutor import execute_program, print_graph_difference

sys.path.append('..')
sys.path.append('../simulation')
from evolving_graph.scripts import Script
import evolving_graph.utils as utils
from evolving_graph.execution import ScriptExecutor
from evolving_graph.environment import EnvironmentGraph


def print_list(l):
  width = max([len(obj) for obj in l]) + 2
  line = ''
  for obj in l:
    line = line + obj.ljust(width,' ')
    if len(line) > 100:
      print(line)
      line = ''
  print(line)

init_graph = GraphReader(init_graph_file)
print(f'\n\nUsing scene {int(scene_num)-1}')
open('program.txt', 'a').close()

print('\n\nYour environment has four rooms: bathroom, dining room, bedroom, home office.')
print('These are all the objects present in your environment, categorized by the rooms.')
for room, stuff in init_graph.usable_nodes_by_room.items():
  if room == 'dining_room':
    print()
    print(('  '+room+'  ').center(100, '#'))
    objs = list(stuff.keys())
    objs.sort()
    print_list(objs)

info = {}
info['scene_num'] = scene_num
info['num_train_routines'] = 15
info['num_test_routines'] = 10
info['dt'] = 10
info['ignored_classes'] = ['wall']
all_states_list = set()

out_dir = 'data/routinesWithStateEmphasis/0720_stretched_breakfast'
if os.path.exists(out_dir):
  shutil.rmtree(out_dir)
os.mkdir(out_dir)



Using scene 1


Your environment has four rooms: bathroom, dining room, bedroom, home office.
These are all the objects present in your environment, categorized by the rooms.

##########################################  dining_room  ###########################################
basket_for_clothes            bench                         bench_dining_room1            bleach                        
blender                       bookshelf                     bookshelf_dining_room         bottlebrush                   
bowl                          broom                         brush                         ceilinglamp_dining_room       
ceilinglamp_dining_room1      chef_knife                    cleaning_bottle               cleaning_solution             
cloth_napkin                  coffe_maker                   coffee                        coffee_cup                    
coffee_filter                 coffee_pot                    colander                      cookingpot                

In [3]:
from evolving_graph.scripts import Action

print('To do this activity, you can use any of the following actions : \n')
from evolving_graph import scripts
action_template = [('['+a.title()+']'+' <.>'*scripts.Action[a].value[1]) for a in scripts.Action.__members__]

print_list(action_template)

To do this activity, you can use any of the following actions : 

[Close] <.>          [Drink] <.>          [Find] <.>           [Walk] <.>           [Grab] <.>           
[Idle]               [Lookat] <.>         [Lookat_Short] <.>   [Lookat_Medium] <.>  [Lookat_Long] <.>    
[Open] <.>           [Pointat] <.>        [Putback] <.> <.>    [Putin] <.> <.>      [Putobjback] <.>     
[Run] <.>            [Sit] <.>            [Standup]            [Switchoff] <.>      [Switchon] <.>       
[Touch] <.>          [Turnto] <.>         [Watch] <.>          [Wipe] <.>           [Puton] <.>          
[Putoff] <.>         [Greet] <.>          [Drop] <.>           [Read] <.>           [Lie] <.>            
[Pour] <.> <.>       [Pourall] <.> <.>    [Poursome] <.> <.>   [Type] <.>           [Push] <.>           
[Pull] <.>           [Move] <.>           [Wash] <.>           [Rinse] <.>          [Scrub] <.>          
[Squeeze] <.>        [Plugin] <.>         [Plugout] <.>        [Cut] <.>            [E

You can copy from the above templates into your scipts and populate them with objects or rooms from the above list. Notice that the that the number of angle brackets signify the number of objects required to specify the action, e.g. `Find` has only one object to specify what to find, whereas `Putin` has two objects specifying what object to put and inside what. Some like `StandUp` and `Sit` might not need an object

In [4]:
def get_scripts(activity, n=None):
    scripts_list = []
    directory = os.path.join('data/scriptsWithStateEmphasis')
    available_files = os.listdir(os.path.join(directory,activity))
    if n is None:
        options = list(set([int(a[:2]) for a in available_files]))
        n = random.choice(options)
    available_files = [a for a in available_files if a.startswith('{:02d}'.format(n))]
    for f in available_files:
        prob = int(f[-6:-4])
        scripts_list.append((prob, {'filename': f}))
        info = read_program(os.path.join('data/scriptsWithStateEmphasis',activity,f), init_graph.node_map)
        scripts_list[-1][1].update(info)
    return scripts_list


def time_human(time_mins):
    time_mins = int(round(time_mins))
    mins = time_mins%60
    time_mins = time_mins//60
    hrs = time_mins%24
    time_mins = time_mins//24
    days = time_mins
    h = '{:02d}:{:02d}'.format(hrs,mins)
    if days != 0:
        h = str(days)+'day - '+h
    return h


utilized_object_ids = set()
def update_used_objects(g1,g2):
    edges_removed = [e for e in g1['edges'] if e not in g2['edges']]
    edges_added = [e for e in g2['edges'] if e not in g1['edges']]
    nodes_removed = [n for n in g1['nodes'] if n['id'] not in [n2['id'] for n2 in g2['nodes']]]
    nodes_added = [n for n in g2['nodes'] if n['id'] not in [n2['id'] for n2 in g1['nodes']]]
    for n in nodes_removed:
        utilized_object_ids.add(n['id'])
    for n in nodes_added:
        utilized_object_ids.add(n['id'])
    for e in edges_removed + edges_added:
        if e['relation_type'] in ['INSIDE', 'ON']:
            utilized_object_ids.add(e['from_id'])
            utilized_object_ids.add(e['to_id'])

def update_states(g):
    for n in g['nodes']:
        for s in n['states']:
            all_states_list.add(s)

def trim_graph(g):
    keep_ids = utilized_object_ids
    new_keep_ids = keep_ids
    while new_keep_ids:
        new_keep_ids = []
        for edge in g['edges']:
            if edge['relation_type'] in ['INSIDE', 'ON']:
                if edge['from_id'] in new_keep_ids and edge['to_id'] not in keep_ids:
                    new_keep_ids.add(edge['to_id'])
        keep_ids.update(new_keep_ids)
    trimmed_graph = {'nodes':[], 'edges':[]}
    for n in g['nodes']:
        if n['id'] in keep_ids:
            if n['class_name'] in info['ignored_classes']:
                keep_ids.remove(n['id'])
            else:
                trimmed_graph['nodes'].append(n)
    trimmed_graph['edges'] = [e for e in g['edges'] if e['from_id'] in keep_ids and e['to_id'] in keep_ids]
    return trimmed_graph
    
    
class SamplingFailure(Exception):
    pass

In [5]:
def make_routine(activity_name):
    scripts_list = get_scripts('breakfast', 0)

    probs = np.array([s[0] for s in scripts_list])
    scr_idx = np.random.choice(np.arange(len(scripts_list)), p=probs/sum(probs))
    info = scripts_list[scr_idx][1]

    info['start_time'] = 360
    duration_min, duration_max = info['total_duration_range']
    sample_duration = random.random() * (duration_max - duration_min) + duration_min
    info['end_time'] = info['start_time'] + sample_duration
    all_actions = []
    script_string = ''

    t=0
    start_time = t
    end_time = info['end_time']
    script_string += '{} ({} - {}) \n'.format(activity_name, time_human(start_time), time_human(end_time))
    remaining_min, remaining_max = deepcopy(info['total_duration_range'])
    duration_remaining = deepcopy(end_time - start_time)
    for act_line, act_duration in zip(info['lines'], info['durations']):
        remaining_min -= act_duration[0]
        remaining_max -= act_duration[1]
        sampling_min = max(act_duration[0], duration_remaining - remaining_max)
        sampling_max = min(act_duration[1], duration_remaining - remaining_min)
        d = (random.random() * (sampling_max-sampling_min) + sampling_min)
        duration_remaining -= d
        all_actions.append({'script':act_line, 'time_from':t, 'time_to':t+d, 'time_from_h':time_human(t), 'time_to_h':time_human(t+d), 'name':activity_name+'-'+info['filename']})
        t += d
    # for a in all_actions:
    #     print (a['time_from_h']+' to '+a['time_to_h']+' : '+a['name']+' : '+str(a['script']))
            

    with open (init_graph_file,'r') as f:
        init_graph_dict = json.load(f)
    name_equivalence = utils.load_name_equivalence()
    complete_script_lines = [action['script'] for action in all_actions]
    executor = ScriptExecutor(EnvironmentGraph(init_graph_dict), name_equivalence)
    success, _, graph_list = executor.execute(Script(complete_script_lines), w_graph_list=True)
    if not success:
        script_info = ''
        try:
            for n in range(len(graph_list)-3, len(graph_list)+1):
                script_info += ('\n## Executing '+all_actions[n]['name']+' from '+all_actions[n]['time_from_h']+' to '+all_actions[n]['time_to_h']+' using '+str(all_actions[n]['script']))
        except:
            pass
        raise SamplingFailure('Execution of following failed because {}... {}'.format(executor.info.get_error_string(), script_info))
    
    graphs = [EnvironmentGraph(init_graph_dict).to_dict()]
    times = []

    last_source = None
    for graph, action_info in zip(graph_list[1:], all_actions):
        src = action_info['name']
        if action_info['name'] != last_source:
            script_string += f'\n\n### {src}\n'
            last_source = src
        script_string += '\n' + str(action_info['script'])
        script_string += '\n## {}\n'.format(action_info['time_to_h'])
        # all_rel = [edge['relation_type'] for edge in graph['edges']]
        # if 'HOLDS_RH' in all_rel or 'HOLDS_LH' in all_rel:
        #     continue
        if len(times) > 0 and action_info['time_to']-times[-1] < 1:
            graphs[-1] = graph
        else:
            update_used_objects(graphs[-1], graph)
            update_states(graph)
            graphs.append(graph)
            times.append(action_info['time_to'])

    routine_out = ({'times':times,'graphs':graphs})

    return routine_out, script_string

In [6]:
os.mkdir(os.path.join(out_dir, 'routines_train'))
os.mkdir(os.path.join(out_dir, 'routines_test'))
os.mkdir(os.path.join(out_dir, 'scripts_train'))
os.mkdir(os.path.join(out_dir, 'scripts_test'))


routines_train = {}
routines_test = {}

# pool = multiprocessing.Pool()

def make_and_dump_breakfast_routine(scripts_dir):
    routine_out, script_string = make_routine('breakfast')
    open(scripts_dir, 'w').write(script_string)
    return routine_out

for routine_num in range(info['num_train_routines']):
    # routines_train[routine_num] = pool.apply_async(make_and_dump_breakfast_routine, args=(os.path.join(out_dir,'scripts_train','{:03d}'.format(routine_num)+'.json')))
    routines_train[routine_num] = make_and_dump_breakfast_routine(os.path.join(out_dir,'scripts_train','{:03d}'.format(routine_num)+'.txt'))

for routine_num in range(info['num_test_routines']):
    # routines_test[routine_num] = pool.apply_async(make_and_dump_breakfast_routine, args=(os.path.join(out_dir,'scripts_test','{:03d}'.format(routine_num)+'.json')))
    routines_test[routine_num] = make_and_dump_breakfast_routine(os.path.join(out_dir,'scripts_test','{:03d}'.format(routine_num)+'.txt'))

# pool.close()
# pool.join()



# pool = multiprocessing.Pool()

def postprocess_routines(routine, out_filepath):
    routine['graphs'] = [trim_graph(g) for g in routine['graphs']]
    json.dump(routine, open(out_filepath,'w'))


for routine_num, routine in routines_train.items():
    # pool.apply_async(postprocess_routines, args=(routine, os.path.join(out_dir,'routines_train','{:03d}'.format(routine_num)+'.json')))
    postprocess_routines(routine, os.path.join(out_dir,'routines_train','{:03d}'.format(routine_num)+'.json'))

for routine_num, routine in routines_test.items():
    # pool.apply_async(postprocess_routines, args=(routine, os.path.join(out_dir,'routines_test','{:03d}'.format(routine_num)+'.json')))
    postprocess_routines(routine, os.path.join(out_dir,'routines_test','{:03d}'.format(routine_num)+'.json'))


# pool.close()
# pool.join()

In [7]:
nodes = routines_train[0]['graphs'][0]['nodes']
all_states_list = list(all_states_list)
all_states_list.sort()
with open(os.path.join(out_dir,'classes.json'), 'w') as f:
    json.dump({"nodes":nodes, "states":all_states_list, "edges":['INSIDE','ON']}, f, indent=4)


info['num_nodes'] = len(nodes)
with open(os.path.join(out_dir,'info.json'), 'w') as f:
    json.dump(info, f, indent=4)

